In [1]:
#Load required packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas
import chardet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
page = 146
links_data = []
row = 0

In [ ]:
#Save every link that indicates the presence of a variable containing foodex2 codes and the respective survey name
dietary_data = pandas.DataFrame(columns = ['Survey Name', 'Survey Data Links'])
while page <= 765:

    website = requests.get('https://www.globaldietarydatabase.org/management/microdata-surveys/' + str(page))
    soup = BeautifulSoup(website.content, 'html.parser')
    file_link = soup.find_all('div', class_="field--item")
    foodex2_link = soup.find_all('div', class_='field field--name-field-foodex2 field--type-list-string field--label-above')

    if len(foodex2_link) == 0:
        page = page + 1

    else:
        foodex2_availability = foodex2_link[0].find('div', class_='field--item').get_text()
        dietary_datafile = soup.find_all('div', class_= 'field field--name-field-data-upload field--type-file field--label-above')

        if foodex2_availability == 'Yes' and  len(dietary_datafile) != 0:

            dietary_datafile_link = dietary_datafile[0].find_all('span', class_='file-link')

            for i in dietary_datafile_link:
                if i.find_all('a', href=True)[0]['href'].lower().find('dietdata') >= 0:
                    links_data.append(i.find_all('a', href=True)[0]['href'])

            dietary_data.at[row , 'Survey Name'] = soup.find('h1', class_="page-header").getText().rstrip()
            dietary_data.at[row , 'Survey Data Links'] = links_data
            page = page + 1
            row = row + 1
            links_data = []

        else:
            page = page + 1

In [ ]:
#Save Edited Results
'''dietary_data.to_csv(path_or_buf = '/content/drive/My Drive/dietary_data_2023-05-17.csv', index= False)

In [ ]:
dietary_data

,Survey Name,Survey Data Links
0,Swedish National Dietary Survey – Riksmaten vu...,[https://www.globaldietarydatabase.org/system/...
1,Mabat First Israeli National Health and Nutrit...,[https://www.globaldietarydatabase.org/system/...
2,Mabat Youth - First Israeli National Health an...,[https://www.globaldietarydatabase.org/system/...
3,Mabat Zahav - National Health and Nutrition Su...,[https://www.globaldietarydatabase.org/system/...
4,Malaysia Lipid Study,[https://www.globaldietarydatabase.org/system/...
5,Food Consumption and Iron Status Survey in the...,[https://www.globaldietarydatabase.org/system/...
6,National Dietary Survey (The Baltic Project) (...,[https://www.globaldietarydatabase.org/system/...
7,National Food Consumption Survey in Italy (INR...,[https://www.globaldietarydatabase.org/system/...
8,"6th National Nutrition Survey, Philippines",[]
9,"Dietary Behavior, Food and Nutrient Intake for...",[https://www.globaldietarydatabase.org/system/...


In [ ]:
#Load dataset with Survey Names and respective URLs
dietary_data_links = pd.read_csv('/content/drive/My Drive/FoodEx2 project/Food Data/dietary_data.csv', header = 0)

#Create a dataframe to group all the data from the different surveys
dietary_data_descriptions = pandas.DataFrame(columns = ['survey_name', 'rcp_ingr_label', 'original_id', 'food_descr_ori',
                                                        'food_descr_eng', 'foodex2_code', 'foodex2_descr'])

x = 0
#Go trough all the surveys gatheres from the GGD website
while x < len(dietary_data_links['Survey Data Links']):

  #For all the links associated with each survey gather the data and save it in the predefined file
  for y in eval(dietary_data_links['Survey Data Links'][x]):

    #Verify if the file is csv or xlsx
    if y[-3:] == 'csv':

      #Read and save the data provided by the link
      dietary_data_temporary = pd.read_csv(filepath_or_buffer = y, header = 0)

    if y[-4:] == 'xlsx':
      dietary_data_temporary = pd.read_excel(io = y, header = 0)

    #If it is the first time adding data to the predefined dataframe:
    if len(dietary_data_descriptions) == 0:

      #Save initial size of the dataframe
      earlier_size = len(dietary_data_descriptions)

      #Get the descriptions for the rcp and other corresponding informations
      dietary_data_descriptions = dietary_data_descriptions.assign(original_id = dietary_data_temporary.loc[~dietary_data_temporary['rcp_descr'].isna(), 'id'],
                                                              food_descr_ori = dietary_data_temporary.loc[~dietary_data_temporary['rcp_descr'].isna(), 'rcp_descr'],
                                                              food_descr_eng = dietary_data_temporary.loc[~dietary_data_temporary['rcp_descr'].isna(), 'rcp_descr_eng'],
                                                              foodex2_code = dietary_data_temporary.loc[~dietary_data_temporary['rcp_descr'].isna(), 'foodex2_rcp_code'],
                                                              foodex2_descr = dietary_data_temporary.loc[~dietary_data_temporary['rcp_descr'].isna(), 'foodex2_rcp_descr'],
                                                              survey_name = str(dietary_data_links['Survey Name'][x]),
                                                              rcp_ingr_label = 'rcp')

      dietary_data_descriptions = dietary_data_descriptions.reset_index(drop = True)

      #Get the descriptions for the ingr and other corresponding informations

      #Create a temporary dataframe to save the descriptions
      dietary_data_temporary_sub = dietary_data_temporary.loc[~dietary_data_temporary['ingr_descr'].isna(), ['id', 'ingr_descr', 'ingr_descr_eng', 'foodex2_ingr_code', 'foodex2_ingr_descr']]

      #Identify from which survey the descritions come from and what they describe, i.e. rcp or ingr
      dietary_data_temporary_sub['survey_name'] = str(dietary_data_links['Survey Name'][x])
      dietary_data_temporary_sub['rcp_ingr_label'] = 'ingr'

      #Rename the columns of the temporary dataframe so it is later corretly appended
      dietary_data_temporary_sub.rename({'id': 'original_id', 'ingr_descr':'food_descr_ori', 'ingr_descr_eng':'food_descr_eng',
                                         'foodex2_ingr_code':'foodex2_code', 'foodex2_ingr_descr':'foodex2_descr'},  axis=1, inplace=True)

      #Append the data and reset the index
      dietary_data_descriptions = dietary_data_descriptions.append(dietary_data_temporary_sub)
      dietary_data_descriptions = dietary_data_descriptions.reset_index(drop = True)

      #Verify if the rows containing descriptions where correctly added
      assert len(dietary_data_descriptions) - earlier_size == len(dietary_data_temporary.loc[~dietary_data_temporary['rcp_descr'].isna()]) + len(dietary_data_temporary.loc[~dietary_data_temporary['ingr_descr'].isna()]), 'oops the size aint adding up'

    #If it is not the first time adding data to the predefined dataframe:
    else:

      #Save initial size of the dataframe
      earlier_size = len(dietary_data_descriptions)

      #Get the descriptions for the rcp and other corresponding informations
      dietary_data_temporary_sub = dietary_data_temporary.loc[~dietary_data_temporary['rcp_descr'].isna(), ['id', 'rcp_descr', 'rcp_descr_eng', 'foodex2_rcp_code', 'foodex2_rcp_descr']]
      dietary_data_temporary_sub['survey_name'] = str(dietary_data_links['Survey Name'][x])
      dietary_data_temporary_sub['rcp_ingr_label'] = 'rcp'

      dietary_data_temporary_sub.rename({'id': 'original_id', 'rcp_descr':'food_descr_ori', 'rcp_descr_eng':'food_descr_eng',
                                         'foodex2_rcp_code':'foodex2_code', 'foodex2_rcp_descr':'foodex2_descr'},  axis=1, inplace=True)

      dietary_data_descriptions = dietary_data_descriptions.append(dietary_data_temporary_sub)
      dietary_data_descriptions = dietary_data_descriptions.reset_index(drop = True)

      #Get the descriptions for the ingr and other corresponding informations
      dietary_data_temporary_sub = dietary_data_temporary.loc[~dietary_data_temporary['ingr_descr'].isna(), ['id', 'ingr_descr', 'ingr_descr_eng', 'foodex2_ingr_code', 'foodex2_ingr_descr']]
      dietary_data_temporary_sub['survey_name'] = str(dietary_data_links['Survey Name'][x])
      dietary_data_temporary_sub['rcp_ingr_label'] = 'ingr'

      dietary_data_temporary_sub.rename({'id': 'original_id', 'ingr_descr':'food_descr_ori', 'ingr_descr_eng':'food_descr_eng',
                                         'foodex2_ingr_code':'foodex2_code', 'foodex2_ingr_descr':'foodex2_descr'},  axis=1, inplace=True)

      dietary_data_descriptions = dietary_data_descriptions.append(dietary_data_temporary_sub)
      dietary_data_descriptions = dietary_data_descriptions.reset_index(drop = True)

      #Verify if the rows containing descriptions where correctly added
      assert len(dietary_data_descriptions) - earlier_size == len(dietary_data_temporary.loc[~dietary_data_temporary['rcp_descr'].isna()]) + len(dietary_data_temporary.loc[~dietary_data_temporary['ingr_descr'].isna()]), 'oops the size aint adding up'

  x = x + 1

In [ ]:
#Save compiled data
'''dietary_data_descriptions.to_csv(path_or_buf = '/content/drive/My Drive/FoodEx2 project/Food Data/dietary_data_descriptions_2023-05-17.csv', index= False)